In [1]:
import torch.nn as nn
import torch.optim as optim
from classes import SpeedEstimatorRNN, VehicleSpeedDataset
from torch.utils.data import DataLoader

In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! You can use a GPU for training.")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU being used:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Training will be performed on the CPU.")

CUDA is available! You can use a GPU for training.
Number of GPUs available: 1
Current GPU being used: 0
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [7]:
# Set dataset path
data_path = r'C:\my files\thesis\AI_training\data\training_data\training_data_noise_extended'
extension = "*.csv"

# Hyperparameters that will alter throughout the model creations
input_size = 26  # Number of CAN signals per timestep
hidden_size = [64, 64, 64, 128, 128, 128, 64, 64, 64, 128, 128, 128]
num_layers = [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2]
learning_rate = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
# num of sequences in one batch
batch_size = [64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64]
dropout_rate = [0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]
sequence_length = [400, 500, 600, 400, 500, 600, 400, 500, 600, 400, 500, 600]


# parameters of the simulation
step_size = 10 # what the overlap between the sequences should look like in the extracted dataset
output_size = 1
num_epochs = 15

num_models = 12

default_location = r"C:\my files\thesis\AI_training\Simple RNN\trained_models\iteration_4\model_"

In [8]:
# Training loops
for j in range(num_models):
    # Load dataset and DataLoader
    dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[j], step_size = step_size)
    dataloader = DataLoader(dataset, batch_size=batch_size[j], shuffle=True, num_workers= 6, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SpeedEstimatorRNN(input_size, hidden_size[j], num_layers[j], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[j])

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (features, speeds) in enumerate(dataloader):
            features, speeds = features.to(device), speeds.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, speeds)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "sequence_length": sequence_length,
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
    }, default_location + str(j) + ".pt")

Epoch [1/15], Loss: 66.9763
Epoch [2/15], Loss: 6.3959
Epoch [3/15], Loss: 1.4663
Epoch [4/15], Loss: 0.8117
Epoch [5/15], Loss: 0.6463
Epoch [6/15], Loss: 0.5716
Epoch [7/15], Loss: 0.5462
Epoch [8/15], Loss: 0.4167
Epoch [9/15], Loss: 0.4270
Epoch [10/15], Loss: 0.3526
Epoch [11/15], Loss: 0.2898
Epoch [12/15], Loss: 0.2786
Epoch [13/15], Loss: 0.2567
Epoch [14/15], Loss: 0.2152
Epoch [15/15], Loss: 0.2226
Epoch [1/15], Loss: 64.9594
Epoch [2/15], Loss: 5.7559
Epoch [3/15], Loss: 2.1266
Epoch [4/15], Loss: 1.0060
Epoch [5/15], Loss: 0.6743
Epoch [6/15], Loss: 0.5558
Epoch [7/15], Loss: 0.4678
Epoch [8/15], Loss: 0.3789
Epoch [9/15], Loss: 0.3480
Epoch [10/15], Loss: 0.3449
Epoch [11/15], Loss: 0.2696
Epoch [12/15], Loss: 0.2202
Epoch [13/15], Loss: 0.2087
Epoch [14/15], Loss: 0.1709
Epoch [15/15], Loss: 0.1569
Epoch [1/15], Loss: 69.3748
Epoch [2/15], Loss: 5.7292
Epoch [3/15], Loss: 1.3461
Epoch [4/15], Loss: 0.6781
Epoch [5/15], Loss: 0.5059
Epoch [6/15], Loss: 0.4214
Epoch [7/15],

In [1]:
#To change the format of the created models
import torch
from classes import SpeedEstimatorRNNModified  # Import your model structure

# Define paths
num_models = 12  # Total number of models
input_size = 14  # Define these based on your training setup
output_size = 1
hidden_size = [64, 64, 64, 128, 128, 256, 128, 128, 128, 128, 128, 128]  # Same as during training
num_layers = [1, 2, 3, 1, 3, 3, 2, 2, 2, 2, 4, 3]  # Model config during training

# Path to existing models and new .pt files
old_model_location = r"C:\work\AI_training\Simple RNN\trained_models\iteration_3\model_"
new_save_location = r"C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_"  # New path to save the .pt files

# Convert each model
for i in range(num_models):
    # Define model architecture
    model = SpeedEstimatorRNNModified(input_size, hidden_size[i], num_layers[i], output_size)

    # Load trained weights
    checkpoint = torch.load(old_model_location + str(i), map_location=torch.device('cpu'))  # Adjust path as needed
    model.load_state_dict(checkpoint["model_state_dict"])  # Load the weights
    model.eval()  # Set model to evaluation mode

    # Save the model in .pt format
    torch.save(model, new_save_location + str(i) + ".pt")
    print(f"Model {i} saved in .pt format at {new_save_location + str(i) + '.pt'}")

Model 0 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_0.pt
Model 1 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_1.pt
Model 2 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_2.pt
Model 3 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_3.pt
Model 4 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_4.pt
Model 5 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_5.pt
Model 6 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_6.pt
Model 7 saved in .pt format at C:\work\AI_training\Simple RNN\trained_models\converted_models\iteration_3\model_7.pt
Model 8 saved in .pt format at C:\work\AI_training\Simple RNN\tr